# Manual Implementation of Neural Network

This is an example of manual implementation of a simple neural network, the network structure is as below:

![jupyter](./network.png)

In [1]:
import math
import numpy.random as rnd

In [2]:
# initialize weights randomly
w0 = [[rnd.rand()] * 3 for i in range(2)]
w1 = [rnd.rand() for i in range(2)]
h = [rnd.rand() for i in range(2)]
print(w0)
print(w1)
print(h)

[[0.14720829484960163, 0.14720829484960163, 0.14720829484960163], [0.6781089854722847, 0.6781089854722847, 0.6781089854722847]]
[0.31659987196981376, 0.5387062821637125]
[0.4590110440278494, 0.5483062242767238]


## Define activation function: sigmoid

In [3]:
from math import e
# define sigmoid
def sigmoid(x):
    return 1/(1+math.pow(e, -1 * 2 * x))

# define sigmoid derivative
def d_sigmoid(x):
    return sigmoid(2*x) * (1 - sigmoid(2*x)) * 2

print(sigmoid(0))
print(d_sigmoid(0))

0.5
0.5


## Define a single neuron 

In [4]:
# define a single neuron
def f(x, w, activate=True):
    ret = 0
    for i in range(len(x)):
        ret += x[i] * w[i]
    if activate:
        return sigmoid(ret)
    else:
        return ret

#print(f([1.0, 0.9, 0.8], w0[0]))

## Define loss function: logloss

In [5]:
# define logloss
epsilon = 0.0000001
def logloss(label, prob):
    return -1 * (label * math.log(prob + epsilon, e) + (1 - label) * math.log(1 - prob + epsilon, e))

# define losloss derivative
def d_logloss(label, prob):
    return -1 * label / (prob+epsilon) + (1 - label)/(1 - prob + epsilon)

print(logloss(1, 0.5))
print(d_logloss(0, 0.01))
print(d_logloss(1, 0.99))

0.6931469805599654
1.0101009080706154
-1.0101009080706154


## Define the sample data
If the value of xi in all dimentions > 0.5, y=1  
If any of the xi <=0.5, y=0

In [6]:
# define training data set x and y
x = [[0.1,0.1, 0.1],
     [0.9, 0.9, 0.9],
     [0.2,0.1,0.1],
     [0.9,0.97,0.89],
     [0.1,0.2,0.1],
     [0.8,0.9,0.9],
     [0.3,0.1,0.4],
     [0.9,0.8,0.7],
     [0.11,0.22,0.15],
     [0.88,0.9,0.9]
   ]
#y = [0, 1]
y = [0, 1, 0, 1, 0, 1, 0, 1, 0, 1]

## Train the neural network

In [12]:
# define feed forward 
def feed_forward(x):
    h[0] = f(x, w0[0])
    h[1] = f(x, w0[1])
    prob = f(h, w1)
    return prob

In [8]:
# define back propagation
# w = w - lr * d_w
def back_propagation(iteration, x, prob, label, LR):
    
    w00_tmp = [w0[0][i] for i in range(len(w0[0]))]
    w01_tmp = [w0[1][i] for i in range(len(w0[1]))]
    w1_tmp = [w1[i] for i in range(len(w1))]
    # update w0[0]
    for i in range(len(w0[0])):
        w0[0][i] = w0[0][i] - LR * d_logloss(label, prob) * ( d_sigmoid(f(h, w1_tmp, False))*w1_tmp[0] ) * ( d_sigmoid(f(x, w00_tmp, False))*x[i] )
        #print("1", label, d_logloss(label, prob) * ( d_sigmoid(f(h, w1_tmp, False))*w1_tmp[0] ) * ( d_sigmoid(f(x, w00_tmp, False))*x[i] ))
    
    # update w0[1]
    for i in range(len(w0[1])):
        w0[1][i] = w0[1][i] - LR * d_logloss(label, prob) * ( d_sigmoid(f(h, w1_tmp, False))*w1_tmp[1] ) * ( d_sigmoid(f(x, w01_tmp, False))*x[i] )
        #print("2", label, d_logloss(label, prob) * ( d_sigmoid(f(h, w1_tmp, False))*w1_tmp[1] ) * ( d_sigmoid(f(x, w01_tmp, False))*x[i] ))

    # update w[1]
    for i in range(len(w1)):
        w1[i] = w1[i] - LR * d_logloss(label, prob) * d_sigmoid(f(h, w1_tmp, False)) * h[i]
        #print("3", label, d_logloss(label, prob) * d_sigmoid(f(h, w1_tmp, False)) * h[i])

In [14]:
# start training
EPOCH = 100
for epo in range(EPOCH):
    loss = 0
    for i in range(len(x)):  # batch size=1
        prob = feed_forward(x[i])
        #print(prob, y[i])
        back_propagation(i, x[i], prob, y[i], 0.4)
        loss += logloss(y[i], prob)
    
    if (epo % 10 == 0):
        print("\nEpoch {}:".format(epo))
        print("w0: " + str(w0))
        print("w1: " + str(w1))
        print("loss:", loss/len(x))
        #print(feed_forward([0.1, 0.1, 0.1]))
        #print(feed_forward([0.9, 0.9, 0.9]))



Epoch 0:
w0: [[-0.48401953716789486, -0.6695132793754524, 0.3286719191832048], [0.38561469596713, 0.41292788243217593, 0.03332288273552692]]
w1: [-7.249274960285876, 3.3274503014616292]
loss: 0.06883330208380216

Epoch 10:
w0: [[-0.4869790145170979, -0.6681500296066486, 0.33032025029581075], [0.38758056630345844, 0.41125769768813625, 0.03304322446467877]]
w1: [-7.314228341542067, 3.3568628639092153]
loss: 0.06724954890365735

Epoch 20:
w0: [[-0.489724016750512, -0.6668816084629198, 0.3318399076616988], [0.3894240232622217, 0.4096686715913526, 0.0328056484012269]]
w1: [-7.376171781467215, 3.384912009528108]
loss: 0.06577319760357474

Epoch 30:
w0: [[-0.4922758076933255, -0.6656997986358532, 0.33324528344659227], [0.39115691452365364, 0.40815405769348473, 0.032605049952269136]]
w1: [-7.4353744100900165, 3.4117190184218904]
loss: 0.064392496510811

Epoch 40:
w0: [[-0.4946529501475716, -0.6645971404162493, 0.33454860217519217], [0.39278957602902165, 0.4067078055560713, 0.03243711368548335

## Let's do some predict :)

In [15]:
# predict 
print(feed_forward([0.9, 0.8, 0.92]))
print(feed_forward([0.11, 0.2, 0.18]))

0.9252002246134885
0.0585646310912279


In [16]:
print(feed_forward([0.9, 0.1, 0.92]))

0.2363558316475044
